In [13]:
#!/usr/bin/env python3
"""
Rhythm Typing Game — Tkinter GUI Version (Pure Python)
Save as: rhythm_typing_gui.py
Run: python rhythm_typing_gui.py
"""

import tkinter as tk
from tkinter import messagebox
import time
import random
import csv
import os
from datetime import datetime

# ---------------------------
# CONFIGURATION
# ---------------------------
GAME_DURATION = 60  # seconds
WORDS = [
    "rhythm", "typing", "python", "keyboard", "speed", "accuracy",
    "timing", "challenge", "practice", "focus", "pattern", "sync",
    "random", "score", "improve", "console", "fun", "creative",
    "simple", "project", "game", "study", "minutes", "seconds",
    "track", "record", "statistics", "learn", "repeat"
]
SCORES_FILE = "scores.csv"


# ---------------------------
# HELPER FUNCTIONS
# ---------------------------
def compute_char_level_correct(target, typed):
    correct = 0
    for a, b in zip(target, typed):
        if a == b:
            correct += 1
    return correct


def save_score_row(row):
    header = ["datetime", "duration_s", "wpm", "accuracy_pct", "correct_words", "total_words", "avg_reaction_s", "composite_score"]
    write_header = not os.path.exists(SCORES_FILE)
    try:
        with open(SCORES_FILE, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            if write_header:
                writer.writerow(header)
            writer.writerow(row)
    except Exception as e:
        print("Error saving score:", e)


# ---------------------------
# MAIN GAME CLASS
# ---------------------------
class RhythmTypingGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Rhythm Typing Game 🎵")
        self.root.geometry("600x400")
        self.root.resizable(False, False)
        self.root.configure(bg="#202020")

        # Stats
        self.start_time = None
        self.words_shown = 0
        self.correct_words = 0
        self.total_typed_chars = 0
        self.correct_typed_chars = 0
        self.reaction_times = []
        self.current_word = ""
        self.word_start_time = 0
        self.remaining_time = GAME_DURATION
        self.game_running = False

        # UI elements
        self.setup_ui()

    # -----------------------
    # UI Setup
    # -----------------------
    def setup_ui(self):
        # Title
        self.title_label = tk.Label(
            self.root, text="🎶 Rhythm Typing Game 🎶",
            font=("Helvetica", 20, "bold"), bg="#202020", fg="#00FFAA"
        )
        self.title_label.pack(pady=10)

        # Timer
        self.timer_label = tk.Label(self.root, text=f"Time: {GAME_DURATION}s", font=("Helvetica", 16), bg="#202020", fg="#FFD700")
        self.timer_label.pack(pady=5)

        # Word display
        self.word_label = tk.Label(self.root, text="", font=("Helvetica", 24, "bold"), bg="#202020", fg="#FFFFFF")
        self.word_label.pack(pady=20)

        # Input field
        self.entry = tk.Entry(self.root, font=("Helvetica", 16), justify="center", state="disabled")
        self.entry.pack(ipadx=20, ipady=5)
        self.entry.bind("<Return>", self.check_word)

        # Stats
        self.stats_label = tk.Label(self.root, text="", font=("Helvetica", 14), bg="#202020", fg="#AAAAAA")
        self.stats_label.pack(pady=15)

        # Button frame
        btn_frame = tk.Frame(self.root, bg="#202020")
        btn_frame.pack(pady=10)

        # Start button
        self.start_button = tk.Button(btn_frame, text="Start Game", font=("Helvetica", 14, "bold"),
                                      bg="#00AAFF", fg="white", command=self.start_game)
        self.start_button.grid(row=0, column=0, padx=8)

        # Quit button
        self.quit_button = tk.Button(btn_frame, text="Quit", font=("Helvetica", 14),
                                     bg="#FF5555", fg="white", command=self.root.destroy)
        self.quit_button.grid(row=0, column=1, padx=8)

    # -----------------------
    # Start game
    # -----------------------
    def start_game(self):
        # Reset stats
        self.start_button.config(state="disabled")
        self.entry.config(state="normal")
        self.entry.delete(0, tk.END)
        self.entry.focus_set()
        self.start_time = time.monotonic()
        self.remaining_time = GAME_DURATION
        self.words_shown = 0
        self.correct_words = 0
        self.total_typed_chars = 0
        self.correct_typed_chars = 0
        self.reaction_times = []
        self.game_running = True
        self.next_word()
        self.update_timer()
        self.stats_label.config(text="WPM: 0.0  |  Accuracy: 0.0%")

    # -----------------------
    # Timer update
    # -----------------------
    def update_timer(self):
        if not self.game_running:
            return

        if self.remaining_time > 0:
            self.timer_label.config(text=f"Time: {int(self.remaining_time)}s")
            self.remaining_time -= 1
            self.root.after(1000, self.update_timer)
        else:
            self.end_game()

    # -----------------------
    # Show next word
    # -----------------------
    def next_word(self):
        if not self.game_running:
            return
        self.current_word = random.choice(WORDS)
        self.word_label.config(text=self.current_word)
        self.word_start_time = time.monotonic()
        self.words_shown += 1
        self.entry.delete(0, tk.END)

    # -----------------------
    # Check word on Enter
    # -----------------------
    def check_word(self, event=None):
        if not self.game_running:
            return

        typed = self.entry.get().strip()
        self.entry.delete(0, tk.END)
        reaction_time = time.monotonic() - self.word_start_time

        # Record stats
        correct_chars = compute_char_level_correct(self.current_word, typed)
        self.total_typed_chars += len(typed)
        self.correct_typed_chars += correct_chars
        self.reaction_times.append(reaction_time)
        if typed == self.current_word:
            self.correct_words += 1

        # Live stats
        acc = (self.correct_typed_chars / self.total_typed_chars * 100) if self.total_typed_chars > 0 else 0.0
        elapsed = time.monotonic() - self.start_time if self.start_time else 0.0
        minutes = max(elapsed / 60.0, 1e-6)
        wpm = (self.correct_typed_chars / 5.0) / minutes
        self.stats_label.config(text=f"WPM: {wpm:.1f}  |  Accuracy: {acc:.1f}%")

        # Next word (only if time remains)
        if self.remaining_time > 0:
            self.next_word()

    # -----------------------
    # End game and show results
    # -----------------------
    def end_game(self):
        self.game_running = False
        self.entry.config(state="disabled")
        total_elapsed = time.monotonic() - self.start_time if self.start_time else 0.0
        minutes = total_elapsed / 60.0 if total_elapsed > 0 else 1e-6
        correct_chars = self.correct_typed_chars
        total_typed = self.total_typed_chars

        wpm = (correct_chars / 5.0) / minutes if minutes > 0 else 0.0
        accuracy_pct = (correct_chars / total_typed * 100.0) if total_typed > 0 else 0.0
        avg_reaction = sum(self.reaction_times) / len(self.reaction_times) if self.reaction_times else 0.0
        baseline_reaction = 1.5
        speed_bonus = max(0.0, (baseline_reaction - avg_reaction) / baseline_reaction) if baseline_reaction > 0 else 0.0
        composite_score = wpm * (accuracy_pct / 100.0) * (1.0 + speed_bonus)

        # Save to CSV
        row = [
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            int(total_elapsed),
            f"{wpm:.2f}",
            f"{accuracy_pct:.2f}",
            self.correct_words,
            self.words_shown,
            f"{avg_reaction:.3f}",
            f"{composite_score:.2f}",
        ]
        save_score_row(row)

        # Show final message
        result_text = (
            f"Game Over!\n\n"
            f"Duration: {total_elapsed:.1f} s\n"
            f"Words Shown: {self.words_shown}\n"
            f"Correct Words: {self.correct_words}\n"
            f"WPM: {wpm:.2f}\n"
            f"Accuracy: {accuracy_pct:.2f}%\n"
            f"Avg Reaction: {avg_reaction:.2f} s\n"
            f"Score: {composite_score:.2f}"
        )

        messagebox.showinfo("Results", result_text)
        # Re-enable Start so user can play again
        self.start_button.config(state="normal")
        self.timer_label.config(text=f"Time: {GAME_DURATION}s")
        self.word_label.config(text="")
        self.stats_label.config(text=f"WPM: {wpm:.1f}  |  Accuracy: {accuracy_pct:.1f}%")
        self.start_time = None


# ---------------------------
# MAIN EXECUTION
# ---------------------------
if __name__ == "__main__":
    root = tk.Tk()
    game = RhythmTypingGame(root)
    root.mainloop()


TclError: no display name and no $DISPLAY environment variable

# Task
Adapt the provided Python script to run in a non-GUI environment, specifically the console.

## Remove tkinter dependencies

### Subtask:
Eliminate all Tkinter-related imports and code, including the `RhythmTypingGame` class's UI setup and the main Tkinter event loop.


**Reasoning**:
The goal is to remove all Tkinter-related code to adapt the script for a console environment. This involves removing the import, UI setup, widget interactions, and the main event loop as per the instructions.



In [15]:
#!/usr/bin/env python3
"""
Rhythm Typing Game — Console Version (Pure Python)
Save as: rhythm_typing_console.py
Run: python rhythm_typing_console.py
"""

import time
import random
import csv
import os
from datetime import datetime

# ---------------------------
# CONFIGURATION
# ---------------------------
GAME_DURATION = 60  # seconds
WORDS = [
    "rhythm", "typing", "python", "keyboard", "speed", "accuracy",
    "timing", "challenge", "practice", "focus", "pattern", "sync",
    "random", "score", "improve", "console", "fun", "creative",
    "simple", "project", "game", "study", "minutes", "seconds",
    "track", "record", "statistics", "learn", "repeat"
]
SCORES_FILE = "scores.csv"


# ---------------------------
# HELPER FUNCTIONS
# ---------------------------
def compute_char_level_correct(target, typed):
    correct = 0
    for a, b in zip(target, typed):
        if a == b:
            correct += 1
    return correct


def save_score_row(row):
    header = ["datetime", "duration_s", "wpm", "accuracy_pct", "correct_words", "total_words", "avg_reaction_s", "composite_score"]
    write_header = not os.path.exists(SCORES_FILE)
    try:
        with open(SCORES_FILE, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            if write_header:
                writer.writerow(header)
            writer.writerow(row)
    except Exception as e:
        print("Error saving score:", e)


# ---------------------------
# MAIN GAME CLASS
# ---------------------------
class RhythmTypingGame:
    def __init__(self):
        # Stats
        self.start_time = None
        self.words_shown = 0
        self.correct_words = 0
        self.total_typed_chars = 0
        self.correct_typed_chars = 0
        self.reaction_times = []
        self.current_word = ""
        self.word_start_time = 0
        self.remaining_time = GAME_DURATION
        self.game_running = False

    # -----------------------
    # Start game
    # -----------------------
    def start_game(self):
        # Reset stats
        print("Starting Rhythm Typing Game...")
        self.start_time = time.monotonic()
        self.remaining_time = GAME_DURATION
        self.words_shown = 0
        self.correct_words = 0
        self.total_typed_chars = 0
        self.correct_typed_chars = 0
        self.reaction_times = []
        self.game_running = True
        self.next_word()
        self.game_loop() # Start the console game loop

    # -----------------------
    # Console game loop
    # -----------------------
    def game_loop(self):
        while self.game_running and self.remaining_time > 0:
            elapsed_since_start = time.monotonic() - self.start_time
            self.remaining_time = GAME_DURATION - elapsed_since_start

            if self.remaining_time <= 0:
                self.end_game()
                break

            # Display word and get input (simplified for console)
            print(f"\nTime: {int(self.remaining_time)}s  |  Type: {self.current_word}")
            typed = input("> ").strip()

            if typed.lower() == "quit": # Allow quitting
                self.end_game()
                break

            self.check_word(typed)
            self.next_word() # Get next word after checking

        if self.remaining_time <= 0:
             self.end_game()


    # -----------------------
    # Show next word
    # -----------------------
    def next_word(self):
        if not self.game_running:
            return
        self.current_word = random.choice(WORDS)
        self.word_start_time = time.monotonic()
        self.words_shown += 1

    # -----------------------
    # Check word
    # -----------------------
    def check_word(self, typed):
        if not self.game_running:
            return

        reaction_time = time.monotonic() - self.word_start_time

        # Record stats
        correct_chars = compute_char_level_correct(self.current_word, typed)
        self.total_typed_chars += len(typed)
        self.correct_typed_chars += correct_chars
        self.reaction_times.append(reaction_time)
        if typed == self.current_word:
            self.correct_words += 1

        # Live stats
        acc = (self.correct_typed_chars / self.total_typed_chars * 100) if self.total_typed_chars > 0 else 0.0
        elapsed = time.monotonic() - self.start_time if self.start_time else 0.0
        minutes = max(elapsed / 60.0, 1e-6)
        wpm = (self.correct_typed_chars / 5.0) / minutes
        print(f"WPM: {wpm:.1f}  |  Accuracy: {acc:.1f}%")


    # -----------------------
    # End game and show results
    # -----------------------
    def end_game(self):
        self.game_running = False
        total_elapsed = time.monotonic() - self.start_time if self.start_time else 0.0
        minutes = total_elapsed / 60.0 if total_elapsed > 0 else 1e-6
        correct_chars = self.correct_typed_chars
        total_typed = self.total_typed_chars

        wpm = (correct_chars / 5.0) / minutes if minutes > 0 else 0.0
        accuracy_pct = (correct_chars / total_typed * 100.0) if total_typed > 0 else 0.0
        avg_reaction = sum(self.reaction_times) / len(self.reaction_times) if self.reaction_times else 0.0
        baseline_reaction = 1.5
        speed_bonus = max(0.0, (baseline_reaction - avg_reaction) / baseline_reaction) if baseline_reaction > 0 else 0.0
        composite_score = wpm * (accuracy_pct / 100.0) * (1.0 + speed_bonus)

        # Save to CSV
        row = [
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            int(total_elapsed),
            f"{wpm:.2f}",
            f"{accuracy_pct:.2f}",
            self.correct_words,
            self.words_shown,
            f"{avg_reaction:.3f}",
            f"{composite_score:.2f}",
        ]
        save_score_row(row)

        # Show final message
        result_text = (
            f"\nGame Over!\n\n"
            f"Duration: {total_elapsed:.1f} s\n"
            f"Words Shown: {self.words_shown}\n"
            f"Correct Words: {self.correct_words}\n"
            f"WPM: {wpm:.2f}\n"
            f"Accuracy: {accuracy_pct:.2f}%\n"
            f"Avg Reaction: {avg_reaction:.2f} s\n"
            f"Score: {composite_score:.2f}"
        )

        print(result_text)


# ---------------------------
# MAIN EXECUTION
# ---------------------------
if __name__ == "__main__":
    game = RhythmTypingGame()
    game.start_game()

Starting Rhythm Typing Game...

Time: 59s  |  Type: focus
> focus
WPM: 15.0  |  Accuracy: 100.0%

Time: 55s  |  Type: python
> python
WPM: 13.3  |  Accuracy: 100.0%

Time: 50s  |  Type: improve
> improve
WPM: 13.6  |  Accuracy: 100.0%

Time: 44s  |  Type: pattern
> pattern
WPM: 15.2  |  Accuracy: 100.0%

Time: 40s  |  Type: python
> python
WPM: 14.7  |  Accuracy: 100.0%

Time: 34s  |  Type: simple
> simple
WPM: 14.8  |  Accuracy: 100.0%

Time: 29s  |  Type: focus
> focus
WPM: 14.7  |  Accuracy: 100.0%

Time: 25s  |  Type: typing
> typing
WPM: 15.2  |  Accuracy: 100.0%

Time: 22s  |  Type: simple
> simple
WPM: 15.3  |  Accuracy: 100.0%

Time: 17s  |  Type: speed
> speed
WPM: 15.5  |  Accuracy: 100.0%

Time: 14s  |  Type: accuracy
> accuracy
WPM: 16.3  |  Accuracy: 100.0%

Time: 10s  |  Type: project
> project
WPM: 16.7  |  Accuracy: 100.0%

Time: 6s  |  Type: speed
> speed
WPM: 17.0  |  Accuracy: 100.0%

Time: 4s  |  Type: challenge
> xhallenge
WPM: 17.4  |  Accuracy: 98.9%

Game Over!


## Summary:

### Data Analysis Key Findings

*   The provided Python script was successfully modified to remove all Tkinter-specific elements, including imports, UI setup code, and the GUI event loop.
*   The core game logic (timing, word selection, input checking, and score calculation) was preserved and adapted for a console environment using standard input and output functions.
*   The console version includes basic output for time remaining, the current word to type, and live WPM and accuracy feedback after each word is typed.
*   Game results, including WPM, accuracy, and a composite score, are calculated and displayed in the console at the end of the game.
*   The game now includes a "quit" command to exit early and saves the results to a CSV file.

### Insights or Next Steps

*   Consider adding visual elements to the console output (e.g., clearing the screen between words, using colors if the console supports it) to improve the user experience in the terminal.
*   Explore using libraries like `curses` (for Unix-like systems) or `colorama` (cross-platform) to create a more dynamic and visually appealing console interface without relying on full GUI toolkits.
